# Notebook pour la Résolution de Sudoku avec DancingLinks

Ce notebook présentera deux approches pour résoudre des puzzles de Sudoku en utilisant l'algorithme Dancing Links (DLX). Nous explorerons d'abord une solution basée sur la bibliothèque `DlxLib`, puis une solution optimisée développée from scratch.

## 1. Introduction à Dancing Links

Dancing Links (DLX) est une technique efficace pour résoudre des problèmes exact cover, popularisée par Donald Knuth. Elle est souvent utilisée pour des problèmes comme le Sudoku, où l'objectif est de couvrir toutes les contraintes avec un ensemble de solutions possibles.

**Références :**
- [Dancing Links Algorithm](https://en.wikipedia.org/wiki/Dancing_Links)
- [DLXLib Documentation](https://github.com/tomerfiliba/dlx)

## 2. Configuration de l'environnement

Installez les packages nécessaires pour ce notebook :

In [19]:
#r "nuget: DlxLib"

Installed Packages DlxLib, 1.3.0

### Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [20]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


Installed Packages XPlot.Plotly.Interactive, 4.1.0

Loading extensions from `C:\Users\Administrateur.000\.nuget\packages\xplot.plotly.interactive\4.1.0\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger  des grilles de Sudoku et tester des solvers.

- `GetSudokus` : Renvoie des listes de Sudoku issues de fichiers de 3 difficultés différentes.
- `SolveSudoku` : effectue un test simple d'un solver sur un sudoku donné.
- `TestSolvers` : exécute les tests de performance sur plusieurs solveurs.
- `DisplayResults` : affiche les résultats des tests sous forme de graphiques.



## 3. Implémentation avec DLXLib

Nous allons commencer par implémenter le solver en utilisant la bibliothèque `DlxLib`.

In [21]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using DlxLib;

public class DancingLinkSolver : ISudokuSolver
{
    public SudokuGrid Solve(SudokuGrid s)
    {
        var internalRows = BuildInternalRowsForGrid(s);
        var dlxRows = BuildDlxRows(internalRows);
        var solutions = new Dlx()
            .Solve(dlxRows, d => d, r => r)
            .ToImmutableList();

        return SolutionToGrid(internalRows, solutions.First());
    }
    
    private static IImmutableList<Tuple<int, int, int, bool>> BuildInternalRowsForGrid(SudokuGrid grid)
    {
        var internalRows = new List<Tuple<int, int, int, bool>>();
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                int value = grid.Cells[row, col];
                internalRows.AddRange(BuildInternalRowsForCell(row, col, value));
            }
        }
        return internalRows.ToImmutableList();
    }

    private static IImmutableList<Tuple<int, int, int, bool>> BuildInternalRowsForCell(int row, int col, int value)
    {
        if (value >= 1 && value <= 9)
        {
            return ImmutableList.Create(Tuple.Create(row, col, value, true));
        }
        else
        {
            var internalRows = new List<Tuple<int, int, int, bool>>(9);
            for (int v = 1; v <= 9; v++)
            {
                internalRows.Add(Tuple.Create(row, col, v, false));
            }
            return internalRows.ToImmutableList();
        }
    }

    private static IImmutableList<IImmutableList<int>> BuildDlxRows(IEnumerable<Tuple<int, int, int, bool>> internalRows)
    {
        var dlxRows = new List<IImmutableList<int>>();
        foreach (var internalRow in internalRows)
        {
            dlxRows.Add(BuildDlxRow(internalRow));
        }
        return dlxRows.ToImmutableList();
    }

    private static IImmutableList<int> BuildDlxRow(Tuple<int, int, int, bool> internalRow)
    {
        var row = internalRow.Item1;
        var col = internalRow.Item2;
        var value = internalRow.Item3;
        var box = RowColToBox(row, col);
        var result = new int[4 * 9 * 9];
        result[row * 9 + col] = 1; 
        result[9 * 9 + row * 9 + value - 1] = 1;
        result[2 * 9 * 9 + col * 9 + value - 1] = 1;
        result[3 * 9 * 9 + box * 9 + value - 1] = 1;
        return result.ToImmutableList();
    }

    private static int RowColToBox(int row, int col)
    {
        return row - (row % 3) + (col / 3);
    }

    private static SudokuGrid SolutionToGrid(
        IReadOnlyList<Tuple<int, int, int, bool>> internalRows,
        Solution solution)
    {
        var grid = new int[9, 9];
        foreach (var (row, col, value, _) in solution.RowIndexes.Select(rowIndex => internalRows[rowIndex]))
        {
            grid[row, col] = value;
        }
        return new SudokuGrid { Cells = grid };
    }
}

## 4. Implémentation Optimisée from Scratch

Nous allons maintenant implémenter une version optimisée de l'algorithme DLX.

In [22]:
public class DlxCustomized
{
    class NodeHead : Node
    {
        internal int size;

        public NodeHead() : base(null) { }
    }

    class Node
    {
        internal Node right = null;
        internal Node left = null;
        internal Node up = null;
        internal Node down = null;
        internal NodeHead nodeHead = null;
        
        internal int rowIndex;
        internal int column ;
        
        internal int value;

        public Node(NodeHead t)
        {
            nodeHead = t;
        }
    }
    
    private NodeHead root;
    private bool stop = false;
    private LinkedList<Node> solutions = new LinkedList<Node>();
    private SudokuGrid sudoku;
    
    public DlxCustomized(SudokuGrid sudokuGrid)
    {
        sudoku = sudokuGrid;
        root = new NodeHead();
        root.right = root;
        root.left = root;
    }

    public SudokuGrid Solve()
    {
        Init();
        search();
        foreach(Node node in solutions)
        {
            sudoku.Cells[node.rowIndex, node.column] = node.value;
        }
        return sudoku;
    }
    
    public void Init()
    {
        Node[] tmp = new Node[9 * 9 * 4];
        root = new NodeHead();
        Node currentNode = root;
        
        for (int j = 0; j < 324; j++)
        {
            currentNode.right = new NodeHead();
            currentNode.right.left = currentNode;
            currentNode = currentNode.right;
            currentNode.rowIndex = j;
            currentNode.up = currentNode;
            currentNode.down = currentNode;
            tmp[j] = currentNode;
        }
        currentNode.right = root;
        root.left = currentNode;

        int imatrix = 0;
        for (int i = 0; i < 9; i++)
        {
            for (int j = 0; j < 9; j++)
            {
                int value = sudoku.Cells[i ,j];
                Node tmpRCCNode, tmpRNCNode, tmpCNCNode, tmpBNCNode;
                
                if (value == 0)
                {
                    value = 1;
                    int RCC = 9 * i + j;
                    int RNC = 81 + 9 * i + value - 1;
                    int CNC = 162 + 9 * j + value - 1;
                    int BNC = 243 + ((i / 3) * 3 + j / 3) * 9 + value - 1;
                    int end = imatrix + 9;
                    for (; imatrix < end; imatrix++)
                    {
                        tmpRCCNode = new Node((NodeHead)tmp[RCC].down);
                        tmpRNCNode = new Node((NodeHead)tmp[RNC].down);
                        tmpCNCNode = new Node((NodeHead)tmp[CNC].down);
                        tmpBNCNode = new Node((NodeHead)tmp[BNC].down);
                        tmpRCCNode.rowIndex = i;
                        tmpRCCNode.column = j;
                        tmpRCCNode.value = value;
                        tmpRNCNode.rowIndex = i;
                        tmpRNCNode.column = j;
                        tmpRNCNode.value = value;
                        tmpCNCNode.rowIndex = i;
                        tmpCNCNode.column = j;
                        tmpCNCNode.value = value;
                        tmpBNCNode.rowIndex = i;
                        tmpBNCNode.column = j;
                        tmpBNCNode.value = value++;
                        ((NodeHead)tmp[RCC].down).size++;
                        ((NodeHead)tmp[RNC].down).size++;
                        ((NodeHead)tmp[CNC].down).size++;
                        ((NodeHead)tmp[BNC].down).size++;
                        tmpRCCNode.right = tmpRNCNode;
                        tmpRNCNode.right = tmpCNCNode;
                        tmpCNCNode.right = tmpBNCNode;
                        tmpBNCNode.right = tmpRCCNode;
                        tmpBNCNode.left = tmpCNCNode;
                        tmpCNCNode.left = tmpRNCNode;
                        tmpRNCNode.left = tmpRCCNode;
                        tmpRCCNode.left = tmpBNCNode;
                        tmpRCCNode.up = tmp[RCC];
                        tmpRCCNode.down = tmp[RCC].down;
                        tmp[RCC].down = tmpRCCNode;
                        tmp[RCC] = tmpRCCNode;
                        tmpRNCNode.up = tmp[RNC];
                        tmpRNCNode.down = tmp[RNC].down;
                        tmp[RNC].down = tmpRNCNode;
                        tmp[RNC++] = tmpRNCNode;
                        tmpCNCNode.up = tmp[CNC];
                        tmpCNCNode.down

 = tmp[CNC].down;
                        tmp[CNC].down = tmpCNCNode;
                        tmp[CNC++] = tmpCNCNode;
                        tmpBNCNode.up = tmp[BNC];
                        tmpBNCNode.down = tmp[BNC].down;
                        tmp[BNC].down = tmpBNCNode;
                        tmp[BNC++] = tmpBNCNode;
                    }
                }
                else
                {
                    int RCC = 9 * i + j;
                    int RNC = 81 + 9 * i + value - 1;
                    int CNC = 162 + 9 * j + value - 1;
                    int BNC = 243 + ((i / 3) * 3 + j / 3) * 9 + value - 1;
                    tmpRCCNode = new Node((NodeHead)tmp[RCC].down);
                    tmpRNCNode = new Node((NodeHead)tmp[RNC].down);
                    tmpCNCNode = new Node((NodeHead)tmp[CNC].down);
                    tmpBNCNode = new Node((NodeHead)tmp[BNC].down);
                    tmpRCCNode.rowIndex = i;
                    tmpRCCNode.column = j;
                    tmpRCCNode.value = value;
                    tmpRNCNode.rowIndex = i;
                    tmpRNCNode.column = j;
                    tmpRNCNode.value = value;
                    tmpCNCNode.rowIndex = i;
                    tmpCNCNode.column = j;
                    tmpCNCNode.value = value;
                    tmpBNCNode.rowIndex = i;
                    tmpBNCNode.column = j;
                    tmpBNCNode.value = value;
                    tmpRCCNode.right = tmpRNCNode;
                    tmpRNCNode.right = tmpCNCNode;
                    tmpCNCNode.right = tmpBNCNode;
                    tmpBNCNode.right = tmpRCCNode;
                    tmpBNCNode.left = tmpCNCNode;
                    tmpCNCNode.left = tmpRNCNode;
                    tmpRNCNode.left = tmpRCCNode;
                    tmpRCCNode.left = tmpBNCNode;
                    tmpRCCNode.up = tmp[RCC];
                    tmpRCCNode.down = tmp[RCC].down;
                    tmp[RCC].down = tmpRCCNode;
                    tmp[RCC] = tmpRCCNode;
                    tmpRNCNode.up = tmp[RNC];
                    tmpRNCNode.down = tmp[RNC].down;
                    tmp[RNC].down = tmpRNCNode;
                    tmp[RNC++] = tmpRNCNode;
                    tmpCNCNode.up = tmp[CNC];
                    tmpCNCNode.down = tmp[CNC].down;
                    tmp[CNC].down = tmpCNCNode;
                    tmp[CNC++] = tmpCNCNode;
                    tmpBNCNode.up = tmp[BNC];
                    tmpBNCNode.down = tmp[BNC].down;
                    tmp[BNC].down = tmpBNCNode;
                    tmp[BNC++] = tmpBNCNode;
                    imatrix++;
                }
            }
        }
    }
    
    public void search()
    {
        if (root.right == root)
        {
            stop = true;
            return;
        }

        NodeHead selected = (NodeHead)root.right;
        int c = selected.size;
        for (NodeHead currentNode = (NodeHead)root.right; currentNode != root; currentNode = (NodeHead)currentNode.right)
        {
            if (c > currentNode.size)
            {
                c = currentNode.size;
                selected = currentNode;
            }
        }

        cover(selected);

        for (Node iNode = selected.down; iNode != selected; iNode = iNode.down)
        {
            solutions.AddLast(iNode);
            for (Node jNode = iNode.right; jNode != iNode; jNode = jNode.right)
            {
                cover(jNode.nodeHead);
            }
            search();
            if (stop)
                return;
            solutions.RemoveLast();
            for (Node jNode = iNode.left; jNode != iNode; jNode = jNode.left)
            {
                uncover(jNode.nodeHead);
            }
        }
        uncover(selected);
        return;
    }

    private void cover(NodeHead node)
    {
        node.left.right = node.right;
        node.right.left = node.left;
        for (Node iNode = node.down; iNode != node; iNode = iNode.down)
        {
            for (Node jNode = iNode.right; jNode != iNode; jNode = jNode.right)
            {
                jNode.up.down = jNode.down;
                jNode.down.up = jNode.up;
                jNode.nodeHead.size--;
            }
        }
    }

    private void uncover(NodeHead node)
    {
        for (Node iNode = node.down; iNode != node; iNode = iNode.down)
        {
            for (Node jNode = iNode.right; jNode != iNode; jNode = jNode.right)
            {
                jNode.up.down = jNode;
                jNode.down.up = jNode;
                jNode.nodeHead.size++;
            }
        }
        node.left.right = node;
        node.right.left = node;
    }
}

### Implémentation du nouveau solver

In [23]:
public class DancingLinkSolverWithCustomDlx : ISudokuSolver
{
    
       
    public SudokuGrid Solve(SudokuGrid sudoku)
    {
        DlxCustomized dlxCustomized = new DlxCustomized(sudoku);
        return dlxCustomized.Solve();
    }
}

#### 5. Comparaison des Performances

Nous allons maintenant comparer les performances des deux approches sur des puzzles de différentes difficultés.

In [24]:
using System.Diagnostics;
using System.Threading;
using System.Threading.Tasks;

var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    ("Dancing Link Solver (DLXLib)", new DancingLinkSolver()),
    ("Dancing Link Solver (Customized)", new DancingLinkSolverWithCustomDlx())
};

var results = SudokuHelper.TestSolvers(solvers);

// Affichage des résultats
foreach (var result in results)
{
    Console.WriteLine($"{result.SolverName} | Difficulty: {result.Difficulty} | Time: {result.Time} ms | Status: {result.Status}");
}

SudokuHelper.DisplayResults(results);

Testing Dancing Link Solver (Customized) on Hard sudokus...

Dancing Link Solver (DLXLib) | Difficulty: Easy | Time: 115,4546 ms | Status: Success
Dancing Link Solver (DLXLib) | Difficulty: Medium | Time: 153,7937 ms | Status: Success
Dancing Link Solver (DLXLib) | Difficulty: Hard | Time: 174,9278 ms | Status: Success
Dancing Link Solver (Customized) | Difficulty: Easy | Time: 9,329 ms | Status: Success
Dancing Link Solver (Customized) | Difficulty: Medium | Time: 6,9385 ms | Status: Success
Dancing Link Solver (Customized) | Difficulty: Hard | Time: 9,7907 ms | Status: Success


<!DOCTYPE html>

<!DOCTYPE html>

<!DOCTYPE html>